In [9]:
runs = 4

import os
import pandas as pd

dataset_path = "/workspaces/evals/dataset"
system_input = None

def battle(system_input1, user_input1, completion1, system_input2, user_input2, completion2):

    battle_path = os.path.join(os.getcwd(), "registry", "data", "battle")
    os.makedirs(battle_path, exist_ok=True)
    battle_path = os.path.join(battle_path, "samples.jsonl")

    # Data
    if system_input1 == None:
        input1 = ([{"role":"user","content":user_input1}])
    else:
        input1 = ([{"role":"system","content":system_input1},{"role":"user","content":user_input1}])

    if system_input2 == None:
        input2 = ([{"role":"user","content":user_input2}])
    else:
        input2 = ([{"role":"system","content":system_input2},{"role":"user","content":user_input2}])

    dataset = [{"input1": input1, "completion1": completion1, "input2": input2, "completion2":completion2}]
    pd.DataFrame(dataset).to_json(battle_path, orient="records", lines=True)
    
    # Run eval
    !oaieval gpt-3.5-turbo battle --record_path /workspaces/evals/evallogs/logs
    
    # Report
    with open("/workspaces/evals/evallogs/logs", "r") as f:
        events_df = pd.read_json(f, lines=True)

    record_path = "/workspaces/evals/evallogs/eval"
    os.makedirs(record_path, exist_ok=True)
    events_df.to_json(os.path.join(record_path, "events"), lines=True, orient="records")
    events_df["data"].to_json(os.path.join(record_path, "data"), lines=True, orient="records")
    data_df = pd.read_json(os.path.join(record_path, "data"), lines=True)
    for row in data_df[0]:
        s = str(row)
        s = s.replace("[","",-1)
        s = s.replace("]","",-1)
        if s.startswith("{'choice':"):
            s = s.split("'")[3]
            choice = s
        if s.startswith("{'prompt': {'role': 'user', 'content': 'You are comparing two responses to the following two instructions."):
            sampled = s.split("\\n\\nReasoning:\'}, \'sampled\': ")[1].replace("\\'","'").replace("\\n","\n")
    
    data = {'Choice': choice, 'Instruction1': user_input1, 'Response1': completion1, 'Instruction2': user_input2, 'Response2': completion2, 'Sampled': sampled}
    
    return data


def test_match(test_dataset):
    
    data = []
    for t in range(0,len(test_dataset)):
        data.append({"input":[{"role":"system","content":""},{"role":"user","content":test_dataset[t][0]}], "ideal":test_dataset[t][2]})
    df = pd.DataFrame(data)

    # Paths. Assuming this notebook is in evals/evals

    test_match_path = os.path.join(os.getcwd(), "registry", "data", "match")
    os.makedirs(test_match_path, exist_ok=True)
    test_match_path = os.path.join(test_match_path, "samples.jsonl")

    df.to_json(test_match_path, orient="records", lines=True)
    
    !oaieval gpt-3.5-turbo match --record_path /workspaces/evals/evallogs/logs
    
    events = "/workspaces/evals/evallogs/logs"
    record_path = os.path.join("/workspaces/evals/evallogs/eval")
    with open(events, "r") as f:
        events_df = pd.read_json(f, lines=True)
    
    os.makedirs(record_path, exist_ok=True)
    events_df.to_json(os.path.join(record_path, "events"), lines=True, orient="records")
    events_df["data"].to_json(os.path.join(record_path, "data"), lines=True, orient="records")
    data_df = pd.read_json(os.path.join(record_path, "data"), lines=True)
    
    return data_df

def content_to_list(content, dataset):
    dataset_df = pd.DataFrame(dataset)
    return dataset[dataset_df[0].to_list().index(content)]

def append_response(content, response, dataset):
    dataset_df = pd.DataFrame(dataset)
    index = dataset_df[0].to_list().index(content)
    dataset[index][2].append(response)
    return len(dataset[index][2])
dataset_df = pd.read_json(os.path.join(dataset_path, "dataset"), lines=True)
test_dataset_df = pd.read_json(os.path.join(dataset_path, "test_dataset"), lines=True)
best_response_df = pd.read_json(os.path.join(dataset_path, "best_response"), lines=True)
best_response_input_df = pd.read_json(os.path.join(dataset_path, "best_response_input"), lines=True)
battles_df = pd.read_json(os.path.join(dataset_path, "battles"), lines=True)
n_responses_df = pd.read_json(os.path.join(dataset_path, "n_responses"), lines=True)
test_match_dataset_df = pd.read_json(os.path.join(dataset_path, "test_match_dataset"), lines=True)

dataset = dataset_df.values.tolist()
test_dataset = test_dataset_df.values.tolist()
best_response = best_response_df.values.tolist()
best_response_input = best_response_input_df.values.tolist()
n_responses = n_responses_df.values.tolist()


for i in range(runs):

    test_match_data = test_match(test_dataset)
    test_match_dataset_df = pd.concat([test_match_dataset_df, test_match_data])

    same_best_response = True
    for i in range(3,len(test_match_data[0]),2):
        if test_match_data[0][i]["correct"] == False:
            sampled = test_match_data[0][i]["sampled"]
            if sampled not in dataset_df[0].to_list():
                content = test_match_data[0][i-1]["prompt"][1]["content"]
                dataset.append([sampled,content,[]])
                dataset_df = pd.DataFrame(dataset)
                n = append_response(content, sampled, dataset)
                n_responses.append([n])

                data = battle(system_input,content,sampled,system_input,best_response_input[-1][0],best_response[-1][0]) 
                battles_df = pd.concat([battles_df,pd.DataFrame(data, index=[0])], ignore_index=True)

                if data["Choice"] == "Yes":
                    best_response_input.append([content])
                    best_response.append([sampled])
                    test_dataset = [content_to_list(content,dataset)]
                    test_dataset_df = pd.DataFrame(test_dataset)
                    same_best_response = False

    if same_best_response:
        test_dataset0 = test_dataset.copy()
        for t in test_dataset0:
            if t[0] != "":
                if t[1] not in test_dataset_df[0].to_list():
                    test_dataset.append(content_to_list(t[1],dataset))
                    test_dataset_df = pd.DataFrame(test_dataset)
            l = content_to_list(t[0], dataset)
            for response in l[2]:
                if response not in test_dataset_df[0].to_list():
                    test_dataset.append(content_to_list(response,dataset))
                    test_dataset_df = pd.DataFrame(test_dataset)

    dataset_df = pd.DataFrame(dataset)
    test_dataset_df = pd.DataFrame(test_dataset)
    best_response_df = pd.DataFrame(best_response)
    best_response_input_df = pd.DataFrame(best_response_input)
    n_responses_df = pd.DataFrame(n_responses)  
    
    dataset_df.to_json(os.path.join(dataset_path, "dataset"), lines=True, orient="records")
    test_dataset_df.to_json(os.path.join(dataset_path, "test_dataset"), lines=True, orient="records")
    best_response_df.to_json(os.path.join(dataset_path, "best_response"), lines=True, orient="records")
    best_response_input_df.to_json(os.path.join(dataset_path, "best_response_input"), lines=True, orient="records")
    n_responses_df.to_json(os.path.join(dataset_path, "n_responses"), lines=True, orient="records")

    pd.DataFrame(dataset_df[0]).to_json(os.path.join(dataset_path, "dataset0"), lines=True, orient="records")
    pd.DataFrame(dataset_df[1]).to_json(os.path.join(dataset_path, "dataset1"), lines=True, orient="records")

    battles_df.to_json(os.path.join(dataset_path, "battles"), lines=True, orient="records")
    test_match_dataset_df.to_json(os.path.join(dataset_path, "test_match_dataset"), lines=True, orient="records")

    responses = []
    for responses_list in dataset_df[2]:
        responses.append(len(responses_list))
        pd.Series(responses)
    responses_df = pd.DataFrame({"len":responses, "responses": dataset_df[2]})
    responses_df.to_json(os.path.join(dataset_path, "responses"), lines=True, orient="records")

    best_responses_battles_df = battles_df[battles_df["Choice"]=="Yes"]
    pd.DataFrame(best_responses_battles_df["Sampled"]).to_json(os.path.join(dataset_path, "best_responses_battles_sampled"), lines=True, orient="records")
    

/tmp/ipykernel_1833/1397368638.py:98: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  battles_df = pd.read_json(os.path.join(dataset_path, "battles"), lines=True)
/tmp/ipykernel_1833/1397368638.py:98: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  battles_df = pd.read_json(os.path.join(dataset_path, "battles"), lines=True)
/tmp/ipykernel_1833/1397368638.py:98: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will

[2023-07-28 09:34:05,605] [registry.py:270] Loading registry from /workspaces/evals/evals/registry/evals
[2023-07-28 09:34:06,331] [registry.py:270] Loading registry from /home/codespace/.evals/evals
[2023-07-28 09:34:06,332] [oaieval.py:138] Run started: 230728093406R64KPFAI
[2023-07-28 09:34:06,334] [data.py:83] Fetching match/samples.jsonl
[2023-07-28 09:34:06,334] [eval.py:33] Evaluating 8 samples
[2023-07-28 09:34:06,338] [eval.py:139] Running in threaded mode with 10 threads!
100%|█████████████████████████████████████████████| 8/8 [00:02<00:00,  2.95it/s]
[2023-07-28 09:34:09,070] [record.py:341] Final report: {'accuracy': 0.625, 'boostrap_std': 0.183983694929741}. Logged to /workspaces/evals/evallogs/logs
[2023-07-28 09:34:09,070] [oaieval.py:177] Final report:
[2023-07-28 09:34:09,070] [oaieval.py:179] accuracy: 0.625
[2023-07-28 09:34:09,070] [oaieval.py:179] boostrap_std: 0.183983694929741
[2023-07-28 09:34:09,074] [record.py:330] Logged 16 rows of events to /workspaces/evals

In [18]:
test_match_data

,0
0,None
1,"{'prompt': [{'role': 'system', 'content': ''},..."
2,"{'correct': True, 'expected': 'Thank you so mu..."
3,"{'prompt': [{'role': 'system', 'content': ''},..."
4,"{'correct': False, 'expected': [], 'picked': N..."
...,...
127,"{'correct': True, 'expected': ['Thank you so m..."
128,"{'prompt': [{'role': 'system', 'content': ''},..."
129,"{'correct': False, 'expected': [], 'picked': N..."
130,"{'prompt': [{'role': 'system', 'content': ''},..."
